In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
from requests.exceptions import HTTPError
import yaml

## Read CDSE credentials

In [ ]:
cdse_credentials_file = "credentials/credentials_cdse.yaml"

with open(cdse_credentials_file, "r") as file:
    cdse_credentials = yaml.safe_load(file)

## Set up access and refresh token

In [ ]:
cdse_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"

data = {
    "grant_type": "password",
    "username": f"{cdse_credentials["login"]}",
    "password": f"{cdse_credentials["password"]}",
    "client_id": "cdse-public",
}

response = requests.post(
    url=cdse_token_url,
    data=data,
).json()

access_token = response["access_token"]
refresh_token = response["refresh_token"]

### Look at contents of response

Importantly, this shows that the authorisation token expires after 600 seconds (10 minutes) and the refresh token expires after 3600 seconds (1 hour).

In [ ]:
response

### Define function to refresh access token
The refresh token can be used to re-generate the access token without supplying usernames and passwords.
This will work for an hour from when the original access token is generated.

For this notebook, working with Python, the function below will refresh the access token and return a new `headers` string for use with `requests`.

In [ ]:
def refresh_cdse_access_token_header(refresh_token):

    cdse_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"

    data = {
        "grant_type": "refresh_token",
        "refresh_token": f"{refresh_token}",
        "client_id": "cdse-public",
    }

    response = requests.post(
        url=cdse_token_url,
        data=data,
    ).json()

    access_token = response["access_token"]
    headers = {"Authorization": f"Bearer {access_token}"}

    return headers

## PULL subscription

### Create the subscription

The limit of the running Subscriptions (PUSH and PULL) for one user is 1. 
The overall limit of the Subscriptions (running and paused) for one user is 10.

The code will attempt to create a subscription, and return a message if it fails

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

# Create pull subscription
pull_subscription_url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions"

pull_subscription_data = {
    "StageOrder": True,
    "FilterParam": "Collection/Name eq 'SENTINEL-1' and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'IW_SLC__1S')",
    "Priority": 1,
    "Status": "running",
    "SubscriptionEvent": "modified"
}

# Try to create the subscription, raise an error if limit of subscriptions is reached
try:
    subscription_response = requests.post(
        url=pull_subscription_url, 
        json=pull_subscription_data, 
        headers=headers
    )
    subscription_response.raise_for_status()
except HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    message = subscription_response.json()["detail"]["message"]
    print(f"HTTP error message: {message}")

### View the subscription

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

cdse_subscription_info_url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions/Info"

subscription_info_response = requests.get(url=cdse_subscription_info_url, headers=headers).json()
subscription_info_response


The subscription Id is needed for further actions, so we can store this as a variable.

In [ ]:
subscription_id = subscription_info_response[0]["Id"]
subscription_id

### Read subscription

The `$top={n_notifications}` in the url will return the top `n` notifications. 
If not provided, one notification will be read by default.
You can request a maximum of 20 notifications.

The whole response will be kept on your queue for 3 days. 
After 3 days, you will lose the `value` and `ProductName` entries.
`ProductId` is still retained.

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

n_notifications = 3

read_subscription_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions({subscription_id})/Read?$top={n_notifications}"
read_subscription_response = requests.get(url=read_subscription_url, headers=headers).json()

In [ ]:
read_subscription_response

### Acknowledge subscription

Notifications will remain on the queue until acknowledged.
The maximum length of a queue is 100,000 notifications.
If using PULL, notifications must be regularly read and acknowledged to avoid hitting this limit.

Each notification comes with a `AckId`, which can be used to acknowledge the notification.
Using the ‘AckId’ token for a specific notification means acknowledging receipt of the notification for which the `AckId` was assigned, along with all preceding read messages.

In [ ]:
product_ack_ids = [(message["ProductName"], message["AckId"]) for message in read_subscription_response]

product_ack_ids

#### Acknowledge the top product
This should only acknowledge a single product.

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

product_to_acknowledge = product_ack_ids[0][0]
product_ack_token = product_ack_ids[0][1]

print(f"Acknowledging product: {product_to_acknowledge}")

acknowledge_subscription_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions({subscription_id})/Ack?$ackid={product_ack_token}"
acknowledge_subscription_response = requests.post(url=acknowledge_subscription_url, headers=headers)

Note that the response lists that one notification was acknowledged:


In [ ]:
acknowledge_subscription_response.json()

#### Acknowledge the top two products by acknowledging the final item from the read

Acknowledging the third (bottom) product from the original read should acknowledge all remaining preceding notifications.

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

product_to_acknowledge = product_ack_ids[-1][0]
product_ack_token = product_ack_ids[-1][1]

print(f"Acknowledging product: {product_to_acknowledge}")

acknowledge_subscription_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions({subscription_id})/Ack?$ackid={product_ack_token}"
acknowledge_subscription_response = requests.post(url=acknowledge_subscription_url, headers=headers)

Note that the response lists that two notifications were acknowledged:

In [ ]:
acknowledge_subscription_response.json()

## Manage the subscription

### Pause the subscription

Set `Status` to `paused`

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

subscription_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions({subscription_id})"

data = {
    "Status": "paused",
}

pause_subscription_response = requests.patch(url=subscription_url, json=data, headers=headers)

In [ ]:
pause_subscription_response.json()

### Resume the subscription

Set `Status` to `running`

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

subscription_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions({subscription_id})"

data = {
    "Status": "running",
}

running_subscription_response = requests.patch(url=subscription_url, json=data, headers=headers)

In [ ]:
running_subscription_response.json()

### Delete subscription

Can be done in two ways:
* Set `Status` to `cancelled` (same format as above)
* run a `DELETE` request with the subscription ID

We show the second option here.
`<Response [204]>` indicating no content is expected.

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

subscription_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions({subscription_id})"

delete_response = requests.delete(url=subscription_url, headers=headers)

In [ ]:
delete_response